In [ ]:
#| default_exp helpers

In [ ]:
#| export
from __future__ import annotations

# Helpers

> ...

# Prologue

In [ ]:
#| export
import json
import os
import sys
from binascii import hexlify
from functools import reduce
from inspect import Parameter
from typing import Any
from typing import DefaultDict
from typing import Hashable
from typing import Mapping
from typing import overload
from typing import Sequence

import fastcore.all as FC
from fastcore.xml import FT
from fastcore.xml import NotStr
from fasthtml.basics import ft_html
from fasthtml.core import FastHTML
from IPython.display import display
from IPython.display import DisplayHandle


In [ ]:
import operator
from pathlib import Path

import fastcore.all as FC
import ipywidgets as W
from fastcore.test import *
from IPython.display import HTML
from olio.common import setup_console


----

In [ ]:
console, cprint = setup_console(140)


In [ ]:
#| exporti

_n = '\n'

----

# Helpers

# Singleling
> Basic, we're-all-adults-here, singleton.

In [ ]:
#| export

class Singleling:
    def __new__(cls, *args, **kwargs):
        if '__instance__' not in cls.__dict__: cls.__instance__ = super().__new__(cls, *args, **kwargs)
        return cls.__instance__
    
    def setup(self, *args, **kwargs):
        "One-time setup"
        setattr(type(self), 'setup', FC.noop)


In [ ]:
class TestSingle(Singleling):
    def __init__(self):
        self.setup()

test_is(o := TestSingle(), TestSingle())


# _get_globals


In [ ]:
#| exporti

def _get_globals(mod: str):
    if hasattr(sys, '_getframe'):
        glb = sys._getframe(2).f_globals
    else:
        glb = sys.modules[mod].__dict__
    return glb


In [ ]:
def _gtest(): return _get_globals(__name__)
g1 = _gtest()
g2 = globals()
test_eq(g1, g2)


# cleanupwidgets
> Helper to properly cleanup ipywidget instances by closing their comms.

When working with ipywidgets in notebooks, each widget creates a comm channel with the kernel. During development, it's better to to close the widgets, to avoid memory leaks and kernel issues.


In [ ]:
#| export

def cleanupwidgets(*ws, mod: str|None=None, clear=True):
    from IPython.display import clear_output
    glb = _get_globals(mod or __name__)
    if clear: clear_output(wait=True)
    for w in ws:
        _w = glb.get(w) if isinstance(w, str) else w
        if _w:
            try: _w.close()  # type: ignore
            except: pass


In [ ]:
_b = W.Button()
test_eq(_b.comm is not None, True)
cleanupwidgets('_b')
test_is(_b.comm, None)


# Rich display

In [ ]:
def display_json(json):
    from rich.json import JSON
    from rich.jupyter import display as rich_display
    json_renderable = JSON.from_data(json)
    a = list(console.render(json_renderable))
    rich_display(a, console._render_buffer(a))


In [ ]:

display_json({'a': 1, 'b': 2})


{
  "a": 1,
  "b": 2
}

In [ ]:
#| export

@overload
def pretty_repr(*o, html:bool=True, text:bool=False, **kwargs) -> str: ...
@overload
def pretty_repr(*o, html:bool=False, text:bool=True, **kwargs) -> str: ...
def pretty_repr(*o, html:bool=True, text:bool=True, **kwargs) -> dict[str, str]|str:
    from rich.pretty import Pretty
    d = Pretty(*o, **kwargs)._repr_mimebundle_(
        include=((),('text/plain',))[text] + ((),('text/html',))[html], 
        exclude=((),('text/plain',))[not text] + ((),('text/html',))[not html]
        )
    return d if len(d) > 1 else tuple(d.values())[0]


In [ ]:
display(HTML(pretty_repr({'a': 1, 'b': [1,2,3]}, text=False)))
print(pretty_repr({'a': 1, 'b': [1,2,3]}, html=False))
cprint({'a': 1, 'b': [1,2,3]})

{'a': 1, 'b': [1, 2, 3]}



{'a': 1, 'b': [1, 2, 3]}

In [ ]:
#| export

def rich_display(*o, dhdl: DisplayHandle|None=None):
    if not o: return
    vv:tuple[str, ...] = tuple(FC.flatten([_.items() for _ in map(pretty_repr, o)]))  # type: ignore
    dd = {'text/plain':'\n'.join(vv[1::4]), 'text/html':'\n'.join(vv[3::4])}
    if dhdl: dhdl.update(dd, raw=True)
    else: display(dd, raw=True)


In [ ]:
rich_display({'a': 1, 'b': 2}, [3, 4, 5])

{'a': 1, 'b': 2}

[3, 4, 5]

In [ ]:
dhdl = display(display_id=True)
rich_display({'a': 1, 'b': 2}, [3, 4, 5], dhdl=dhdl)


{'a': 1, 'b': 2}

[3, 4, 5]

## CLog

In [ ]:
#| export

def CLog(*o):
    return f"<script>console.log({','.join(map(repr, o))})</script>"


In [ ]:
display(HTML(CLog('aaaa', 'bbbb')))


# Kounter
> Counter of keys

In [ ]:
#| export

class kounter:
    def __init__(self): self.d = DefaultDict(int)
    def __call__(self, k): d = self.d; d[k] += 1; return self.d[k]


In [ ]:
cntr = kounter()
cntr('a')
cntr('b')
cntr('a')
cntr('a')
cntr('b')
cntr('b')
cntr('b')
test_eq(cntr.d, {'a': 3, 'b': 4})
test_eq(cntr('int'), 1)


# id_gen
> Generate unique IDs for HTML elements


In [ ]:
#| export

def simple_id():
    return 'b'+hexlify(os.urandom(16), '-', 4).decode('ascii')

def id_gen():
    kntr = kounter()
    def _(o:Any=None): 
        if o is None: return simple_id()
        return f"{type(o).__name__}_{id(o) if isinstance(o, Hashable) else kntr(type(o).__name__)}"
    return _

The `id_gen` function creates a closure that generates unique IDs by combining random hex strings. Useful for creating unique element IDs in dynamic HTML content.


In [ ]:
new_id = id_gen()
new_id()

'b36234186-b484a060-89053fe5-347c25bf'

In [ ]:
new_id(7)

'int_4302512096'

In [ ]:
new_id(7)

'int_4302512096'

# find
> Lookup values in nested json/mappings/sequences using dot notation


In [ ]:
def find(element, json):
    return reduce(operator.getitem, element.split('.'), json)

j = {"app": {
    "Garden": {
        "Flowers": {
            "Red flower": "Rose",
            "White Flower": "Jasmine",
            "Yellow Flower": "Marigold"
        }
    },
    "Fruits": {
        "Yellow fruit": "Mango",
        "Green fruit": "Guava",
        "White Flower": "groovy"
    },
    "Trees": {
        "label": {
            "Yellow fruit": "Pumpkin",
            "White Flower": "Bogan"
        }
    }
}}

test_eq(find('app.Garden.Flowers.White Flower', j), 'Jasmine')

In [ ]:
cprint((dp := 'app.Garden.Flowers'), reduce(operator.getitem, dp.split('.'), j))


app.Garden.Flowers
{'Red flower': 'Rose', 'White Flower': 'Jasmine', 'Yellow Flower': 'Marigold'}

In [ ]:
def find(element, j:str):
    d = json.loads(j)
    return reduce(lambda d, k: d[k] if isinstance(d, Mapping) else d[int(k)], element.split('.'), d)

j2 = {"app": {
    "Garden": {
        "Flowers": {
            "Red flower": "Rose",
            "White Flower": "Jasmine",
            "Yellow Flower": "Marigold"
        }
    },
    "Fruits": {
        "Yellow fruit": ["Mango", {"Banana": ["Canary Island", "Puerto Rico"]}],
        "Green fruit": "Guava",
        "White Flower": "groovy"
    },
    "Trees": {
        "label": {
            "Yellow fruit": "Pumpkin",
            "White Flower": "Bogan"
        }
    }
}}
test_eq(find('app.Fruits.Yellow fruit.1.Banana.0', json.dumps(j2)), 'Canary Island')


In [ ]:
apollo_astronauts = json.loads(Path('apollo_astronauts.json').read_text())
cprint((dp := 'Apollo 11.Michael Collins'), reduce(operator.getitem, dp.split('.'), apollo_astronauts))


Apollo 11.Michael Collins
{
    'Experience': 'Pilot on Gemini 10 and Command Module pilot on Apollo 11.',
    'Place in history': 'Collins was the first person to perform two EVAs in one mission.',
    'Fast fact': 'Collins says his "secret terror" was returning to Earth alone if the surface mission failed.',
    'Lunar wisdom': 'I really believe that if the political leaders of the world could see their planet from a distance of 100,000 miles 
their outlook could be fundamentally changed. That all-important border would be invisible, that noisy argument silenced.'
}

In [ ]:
#| export

_II = isinstance
def _at(d: Mapping|Sequence, k: str) -> Any:
    return d[k] if _II(d, Mapping) else d[int(k)] if _II(d, Sequence) and not _II(d, (str, bytes)) else None

def find(key_path: str, j: Mapping|Sequence|str|bytes|bytearray, default:Any=Parameter.empty, sep:str='.') -> Any:
    try: return reduce(_at, key_path.split(sep), json.loads(j) if _II(j, (str, bytes, bytearray)) else j)
    except (KeyError, IndexError) as e:
        if default is not Parameter.empty: return default
        raise e


In [ ]:
j2 = {
    "app": {
        "Garden": {
            "Flowers": {
                "Red flower": "Rose",
                "White Flower": "Jasmine",
                "Yellow Flower": "Marigold"
            }
        },
        "Fruits": {
            "Yellow fruit": ["Mango", {"Banana": ["Canary Island", "Puerto Rico"]}],
            "Green fruit": "Guava",
            "White Flower": "groovy"
        },
        "Trees": {
            "label": {
                "Yellow fruit": "Pumpkin",
                "White Flower": "Bogan"
            }
        },
        "Numbers": [1, 2, 3, 4, 5],
        "Boolean": True,
        "Null": None
    }
}

j2_str = json.dumps(j2)

test_eq(find('app.Fruits.Yellow fruit.1.Banana.0', j2_str), 'Canary Island')
test_eq(find('app.Garden.Flowers.Red flower', j2_str), 'Rose')
test_eq(find('app.Numbers.2', j2_str), 3)
test_eq(find('app.Boolean', j2_str), True)
test_eq(find('app.Null', j2_str), None)
test_fail(lambda: find('app.NonExistent', j2_str))
test_fail(lambda: find('app.Fruits.Yellow fruit.3', j2_str))
test_is(find('app.Fruits.Yellow fruit.3', j2_str, None), None)


# read_vfile

Hacking around to get anywidget `vfile:` working in Script and Style. 
Unfortunately, it's not a public API. If useful, we could write a similar util for Bridget.


In [ ]:
#| export

def read_vfile(cts:str)->str|None:
    import anywidget
    from anywidget._file_contents import _VIRTUAL_FILES
    if cts.startswith('vfile:'):
        if fn := _VIRTUAL_FILES.get(cts, None):
            return fn.contents


# vfile: Components

> **FastHTML** xtend `Script` and `Style` with `vfile:` support.

In [ ]:
#| export

@FC.delegates(ft_html, keep=True)  # type: ignore
def Script(code:str="", **kwargs)->FT:
    "A Script tag that doesn't escape its code"
    return ft_html('script', (_n, NotStr(FC.ifnone(read_vfile(code), code))), **kwargs)

@FC.delegates(ft_html, keep=True)  # type: ignore
def Style(*c, **kwargs)->FT:
    "A Style tag that doesn't escape its code"
    return ft_html('style', tuple(NotStr(FC.ifnone(read_vfile(_), _)) for _ in c), **kwargs)


# nb_app
> Basic naked FastHTML app

In [ ]:
#| export

@FC.delegates(FastHTML)  # type: ignore
def nb_app(**kwargs):
    from starlette.middleware.cors import CORSMiddleware
    kwargs.update(default_hdrs=False, sess_cls=None)
    app = FastHTML(**kwargs)
    app.user_middleware = list(filter(lambda x: x.cls is not CORSMiddleware, app.user_middleware))
    return app


# Colophon
----

In [ ]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [ ]:
if FC.IN_NOTEBOOK:
    nb_path = '01_helpers.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
